<a href="https://colab.research.google.com/github/GtoSN6/backtest_lab/blob/main/backtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Install if not already installed
!pip install -q pandas_ta backtrader



In [9]:
from google.colab import files
uploaded = files.upload()


Saving XAUUSD_H1_201001040100_202507252300.csv to XAUUSD_H1_201001040100_202507252300.csv


In [13]:
import pandas as pd

# Replace 'yourfile.csv' with actual file name after upload
df = pd.read_csv('XAUUSD_H1_201001040100_202507252300.csv')
print(df.head())


       <DATE>    <TIME>   <OPEN>   <HIGH>    <LOW>  <CLOSE>  <TICKVOL>  <VOL>  \
0  2010.01.04  01:00:00  1094.61  1096.90  1093.08  1094.49       3364      0   
1  2010.01.04  02:00:00  1094.95  1095.95  1094.00  1095.65       3017      0   
2  2010.01.04  03:00:00  1095.35  1099.15  1095.28  1097.87       3242      0   
3  2010.01.04  04:00:00  1097.87  1099.64  1096.95  1098.89       2677      0   
4  2010.01.04  05:00:00  1098.86  1102.75  1098.52  1101.60       2954      0   

   <SPREAD>  
0         0  
1         0  
2         0  
3         0  
4         0  


In [17]:
import pandas as pd

# Read the file
df = pd.read_csv("XAUUSD_H1_201901040100_202507252300.csv")

# Rename columns (strip < >)
df.columns = [col.strip('<>').capitalize() for col in df.columns]

# Combine Date and Time into one datetime column
df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])

# Drop original Date and Time columns
df.drop(columns=['Date', 'Time'], inplace=True)

# Optional: Drop Vol and Spread if they are all 0
df.drop(columns=['Vol', 'Spread'], inplace=True)

# Set Datetime as index
df.set_index('Datetime', inplace=True)

# Show formatted head
print(df.head())


                        Open     High      Low    Close  Tickvol
Datetime                                                        
2010-01-04 01:00:00  1094.61  1096.90  1093.08  1094.49     3364
2010-01-04 02:00:00  1094.95  1095.95  1094.00  1095.65     3017
2010-01-04 03:00:00  1095.35  1099.15  1095.28  1097.87     3242
2010-01-04 04:00:00  1097.87  1099.64  1096.95  1098.89     2677
2010-01-04 05:00:00  1098.86  1102.75  1098.52  1101.60     2954


In [19]:
import backtrader as bt
import pandas as pd

# 1. Custom Data Feed
class CustomPandasData(bt.feeds.PandasData):
    lines = ('trend',)
    params = (('trend', -1),)

# 2. Strategy Class (you already have this)
class BelugaTrendStrategy(bt.Strategy):
    def __init__(self):
        self.trend = self.datas[0].trend
        self.buy_price = None
        self.trades = []

    def next(self):
        if not self.position:
            if self.trend[0] == 1:  # Buy
                self.buy_price = self.data.close[0]
                self.buy()
                self.trades.append({
                    'type': 'BUY',
                    'entry_datetime': self.data.datetime.datetime(0),
                    'entry_price': self.data.close[0],
                    'entry_tick_volume': self.data.volume[0],
                })

            elif self.trend[0] == -1:  # Sell
                self.buy_price = self.data.close[0]
                self.sell()
                self.trades.append({
                    'type': 'SELL',
                    'entry_datetime': self.data.datetime.datetime(0),
                    'entry_price': self.data.close[0],
                    'entry_tick_volume': self.data.volume[0],
                })

        else:
            # Exit on opposite signal
            if (self.position.size > 0 and self.trend[0] == -1) or \
               (self.position.size < 0 and self.trend[0] == 1):

                self.close()
                self.trades[-1].update({
                    'exit_datetime': self.data.datetime.datetime(0),
                    'exit_price': self.data.close[0],
                    'exit_tick_volume': self.data.volume[0],
                    'pnl': self.data.close[0] - self.buy_price if self.position.size > 0
                            else self.buy_price - self.data.close[0]
                })

    def stop(self):
        # Save trades to CSV
        df = pd.DataFrame(self.trades)
        df.to_csv("beluga_trades.csv", index=False)
        print("Trades saved to beluga_trades.csv")


In [24]:
# 3. Load CSV
df = pd.read_csv('XAUUSD_H1_201001040100_202507252300.csv')

# Rename columns (strip < >) - added this step to match the previous cell
df.columns = [col.strip('<>').capitalize() for col in df.columns]


df['datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
df.set_index('datetime', inplace=True)

# Optional: Drop Vol and Spread if they are all 0 (check if columns exist first)
if 'Vol' in df.columns:
    df.drop(columns=['Vol'], inplace=True)
if 'Spread' in df.columns:
    df.drop(columns=['Spread'], inplace=True)


# Calculate a simple trend indicator (e.g., difference from rolling mean)
window_size = 20 # You can adjust this window size
df['trend'] = df['Close'] - df['Close'].rolling(window=window_size).mean()

# Convert trend to signals (1 for buy, -1 for sell, 0 for no signal)
df['trend'] = df['trend'].apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))


# 4. Feed to Backtrader
data = CustomPandasData(dataname=df)

cerebro = bt.Cerebro()
cerebro.addstrategy(BelugaTrendStrategy)
cerebro.adddata(data)
cerebro.run()

# Print the data after execution
print(df.head())

Trades saved to beluga_trades.csv
                           Date      Time     Open     High      Low    Close  \
datetime                                                                        
2010-01-04 01:00:00  2010.01.04  01:00:00  1094.61  1096.90  1093.08  1094.49   
2010-01-04 02:00:00  2010.01.04  02:00:00  1094.95  1095.95  1094.00  1095.65   
2010-01-04 03:00:00  2010.01.04  03:00:00  1095.35  1099.15  1095.28  1097.87   
2010-01-04 04:00:00  2010.01.04  04:00:00  1097.87  1099.64  1096.95  1098.89   
2010-01-04 05:00:00  2010.01.04  05:00:00  1098.86  1102.75  1098.52  1101.60   

                     Tickvol  trend  
datetime                             
2010-01-04 01:00:00     3364      0  
2010-01-04 02:00:00     3017      0  
2010-01-04 03:00:00     3242      0  
2010-01-04 04:00:00     2677      0  
2010-01-04 05:00:00     2954      0  


In [25]:
import pandas as pd

# Load the saved trades from the CSV file
trades_df = pd.read_csv("beluga_trades.csv")

# Display the trades DataFrame
print(trades_df)

      type       entry_datetime  entry_price  entry_tick_volume  \
0      BUY  2010-01-04 20:00:00      1117.55                NaN   
1      BUY  2010-01-05 13:00:00      1126.70                NaN   
2     SELL  2010-01-05 15:00:00      1120.40                NaN   
3      BUY  2010-01-05 17:00:00      1123.75                NaN   
4     SELL  2010-01-05 19:00:00      1117.50                NaN   
...    ...                  ...          ...                ...   
8389   BUY  2025-07-23 11:00:00      3428.40                NaN   
8390   BUY  2025-07-23 14:00:00      3428.12                NaN   
8391  SELL  2025-07-23 16:00:00      3413.51                NaN   
8392   BUY  2025-07-25 04:00:00      3370.35                NaN   
8393  SELL  2025-07-25 06:00:00      3359.11                NaN   

            exit_datetime  exit_price  exit_tick_volume    pnl  
0     2010-01-05 12:00:00     1121.40               NaN   3.85  
1     2010-01-05 14:00:00     1120.50               NaN  -6.20  


In [49]:
from google.colab import files

# Download the trades_with_indicators.csv file
files.download('trades_with_indicators.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>